<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/MarkupLM/Fine_tune_MarkupLMForTokenClassification_on_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import wandb

run = wandb.init(
    # Set the project where this run will be logged
    project="diplom-2024",
    # Track hyperparameters and run metadata
)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [8]:
# import utils_data
# from utils_data import get_dataset_and_info_for_websites

from transformers import MarkupLMForTokenClassification
import pandas as pd


NameError: name '_C' is not defined

## Обучение 

In [ ]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.cuda.device_count()

1

In [6]:
print('lalal')

lalal


## Prepare dataset

Next, let's load a toy dataset which we'll use to fine-tune MarkupLM on.

The goal for the model is to label nodes of HTML strings with the appropriate class.

In [ ]:
import numpy as np

max_length = 384
doc_stride = 128

def sample_websites(dataset, k, seed):
    all_websites = list(dataset.keys())
    training_websites = np.random.choice(all_websites, size=k, replace=False)
    print(training_websites)

    train = []
    valid = []
    for website in all_websites:
        if website in training_websites:
            # train += split_dataset(dataset[website], max_length, doc_stride)
            train += dataset[website]
        else:
            # valid += split_dataset(dataset[website], max_length, doc_stride)
            valid += dataset[website]
    
    return train, valid

Загружаем и разбиваем датасет

In [ ]:
import json

with open("/home/modis/vera/camera_dataset.json", "r") as file:
    data = json.load(file)



In [ ]:
train_data, valid_data = sample_websites(data, k=5, seed=0)

['compsource' 'thenerds' 'newegg' 'ecost' 'amazon']


In [ ]:
train_data[0]

{'nodes': [['eCOST.com - Samsung 12.2 MegaPixel Compact Digital Camera w/ Dual LCD Displays - Dark Blue - TL205 DARK BLUE',
   '[if IE]>\n        <link rel="stylesheet" href="/Styles.ashx?file=/App_Themes/Default/DetailIE.css" type="text/css" />\n    <![endif]',
   '[if IE 6]>\n        <link rel="stylesheet" href="/Styles.ashx?file=/App_Themes/Default/DetailIE6.css" type="text/css" />\n    <![endif]',
   '[if IE 7]>\n        <link rel="stylesheet" href="/Styles.ashx?file=/App_Themes/Default/DetailIE7.css" type="text/css" />\n    <![endif]',
   '[if IE 8]>\n        <link rel="stylesheet" href="/Styles.ashx?file=/App_Themes/Default/DetailIE8.css" type="text/css" />\n    <![endif]',
   "if(window.opera && window.print) \n        {\n            var linkStyleElement = document.createElement('link');\n            linkStyleElement.rel = 'stylesheet';\n            linkStyleElement.href = '/Styles.ashx?file=/App_Themes/Default/DetailOpera.css';\n            linkStyleElement.type = 'text/css';\n

Посчитаем количество узлов

In [ ]:
def count_label(data):
    count_model = 0
    count_price = 0
    count_manufacturer = 0
    count_other = 0

    for page in data:
        count_model += page['node_labels'][0].count(0)
        count_price += page['node_labels'][0].count(1)
        count_manufacturer += page['node_labels'][0].count(2)
        count_other += page['node_labels'][0].count(3)

    return {'model':count_model, 'price': count_price, 'manufacturer': count_manufacturer, 'other': count_other}
    

In [ ]:
count_label(train_data)

{'model': 4486, 'price': 6137, 'manufacturer': 5757, 'other': 1466590}

## Create PyTorch Dataset

Next, we'll create a regular PyTorch dataset. Each item of the dataset is an HTML string, encoded using MarkupLMProcessor. Note that we initialize the processor with parse_html = False, as we have already parsed the HTML ourselves and we're providing the nodes, xpaths and node labels.

Note that by default, the processor will only label the first token of a given node and label the remaining tokens with -100. you can change this by setting the `only_label_first_subword` attribute of the processor's tokenizer to `False`.

In [ ]:
from torch.utils.data import Dataset
# from datasets import Dataset, DatasetDict, load_dataset


# длина окна
max_length = 384
doc_stride = 128

class SwdeFeature(object):
    def __init__(self,
                #  html_path,
                 input_ids,
                 token_type_ids,
                 attention_mask,
                 xpath_tags_seq,
                 xpath_subs_seq,
                 labels,
                #  involved_first_tokens_pos,
                #  involved_first_tokens_xpaths,
                #  involved_first_tokens_types,
                #  involved_first_tokens_text_en,
                #  involved_first_tokens_text
                 ):
        """
        html_path: indicate which page the feature belongs to
        input_ids: RT
        token_type_ids: RT
        attention_mask: RT
        xpath_tags_seq: RT
        xpath_subs_seq: RT
        labels: RT
        involved_first_tokens_pos: a list, indicate the positions of the first-tokens in this feature
        involved_first_tokens_xpaths: the xpaths of the first-tokens, used to build dict
        involved_first_tokens_types: the types of the first-tokens
        involved_first_tokens_text: the text of the first tokens
        Note that `involved_xxx` are not fixed-length array, so they shouldn't be sent into our model
        They are just used for evaluation
        """
        # self.html_path = html_path
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.xpath_tags_seq = xpath_tags_seq
        self.xpath_subs_seq = xpath_subs_seq
        self.labels = labels
        # self.involved_first_tokens_pos = involved_first_tokens_pos
        # self.involved_first_tokens_xpaths = involved_first_tokens_xpaths
        # self.involved_first_tokens_types = involved_first_tokens_types
        # self.involved_first_tokens_text_en = involved_first_tokens_text_en
        # self.involved_first_tokens_text = involved_first_tokens_text

# class MarkupLMDataset(Dataset):
#     """Dataset for token classification with MarkupLM."""

#     def __init__(self, data, processor=None, max_length=max_length):
#         self.data = data
#         self.processor = processor
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         # first, get nodes, xpaths and node labels
#         item = self.data[idx]
#         nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

#         # provide to processor
#         encoding = self.processor(nodes=nodes, 
#                                   xpaths=xpaths, 
#                                   node_labels=node_labels, 
#                                   return_tensors="pt")
#         print(encoding)

#         # remove batch dimension
#         encoding = {k: v.squeeze() for k, v in encoding.items()}

#         return encoding
        
class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=max_length):
        self.data = data
        self.processor = processor
        self.max_length = max_length

        input_ids = []
        token_type_ids = []
        attention_mask = []
        xpath_tags_seq = []
        xpath_subs_seq = []
        labels = []

        for page in data:
            nodes, xpaths, node_labels = page['nodes'], page['xpaths'], page['node_labels']

            encoding = self.processor(nodes=nodes, 
                                    xpaths=xpaths, 
                                    node_labels=node_labels, 
                                    return_tensors="pt")
            encoding = {k: v.squeeze() for k, v in encoding.items()}
            input_ids.append(encoding['input_ids'])
            token_type_ids.append(encoding['token_type_ids'])
            attention_mask.append(encoding['attention_mask'])
            xpath_tags_seq.append(encoding['xpath_tags_seq'])
            xpath_subs_seq.append(encoding['xpath_subs_seq'])
            labels.append(encoding['labels'])
                                  
        self.tensors = [input_ids, token_type_ids, attention_mask,
                        xpath_tags_seq, xpath_subs_seq, labels]
        
        self.tensors_name = ['input_ids', 'token_type_ids', 'attention_mask',
                        'xpath_tags_seq', 'xpath_subs_seq', 'labels']
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.xpath_tags_seq = xpath_tags_seq
        self.xpath_subs_seq = xpath_subs_seq
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):        
        item = {}
        for tensor, tensor_name in zip(self.tensors, self.tensors_name):
            item[tensor_name] = tensor[idx]


        return item



class SwdeDataset(Dataset):
    def __init__(self,
                 all_input_ids,
                 all_attention_mask,
                 all_token_type_ids,
                 all_xpath_tags_seq,
                 all_xpath_subs_seq,
                 all_labels=None,
                 ):
        '''
        print(type(all_input_ids))
        print(type(all_attention_mask))
        print(type(all_token_type_ids))
        print(type(all_xpath_tags_seq))
        print(type(all_xpath_subs_seq))
        print(type(all_labels))
        raise ValueError
        '''
        self.tensors = [all_input_ids, all_attention_mask, all_token_type_ids,
                        all_xpath_tags_seq, all_xpath_subs_seq]
        # self.tensors_name = ['input_ids', 'token_type_ids', 'attention_mask',
        #                 'xpath_tags_seq', 'xpath_subs_seq', 'labels']
        
        if not all_labels is None:
            self.tensors.append(all_labels)

    def __len__(self):
        return len(self.tensors[0])

    def __getitem__(self, idx):
        # item = {}
        # for tensor, tensor_name in zip(self.tensors, self.tensors_name):
        #     item[tensor_name] = tensor[idx]


        # return item
        return tuple(tensor[idx] for tensor in self.tensors)

In [ ]:
from transformers import MarkupLMTokenizer

def chunk_page(page):
    max_length = 384
    doc_stride = 128
    tokenizer = MarkupLMTokenizer.from_pretrained("microsoft/markuplm-base")

    #dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'xpath_tags_seq', 'xpath_subs_seq', 'labels'])
    all_token_ids_seq = page['input_ids'].tolist()
    all_xpath_tags_seq = page['xpath_tags_seq'].tolist()
    all_xpath_subs_seq = page['xpath_subs_seq'].tolist()
    all_labels_seq = page['labels'].tolist()
    features = []
    start_pos = 0
    flag = False
    padded_xpath_tags_seq = [216] * 50
    padded_xpath_subs_seq = [1001] * 50
    
    # Разбиваем страницу на чанки с пересечением = doc_stride между чанками
    # doc_stride = 128, real_max_token_num = 382
    # chunk 1: start_pos = 0, end_pos = 0 + 382 = 382
    # chink 2: start_pos = 382 - 128 = 254, end_pos = 254 + 382 = 636
    # ... пока end_pos < len(all_token_ids_seq) или пока не встретим пустой чанк 
    
    while True:
        
        # invloved is [ start_pos , end_pos )

        token_type_ids = [0] * max_length  # that is always this

        end_pos = start_pos + max_length
        # add start_pos ~ end_pos as a feature
        
        # all_ - последовательности для каждого токена текста страницы
        # splited_ - последовательности для каждого токена текста чанка
        # splited_ = [CLS] + all_[start_pos:end_pos] + [SEP]
        # Размер splited_ = 1 + 382 + 1 = 384
        splited_token_ids_seq = all_token_ids_seq[start_pos:end_pos] 
        splited_xpath_tags_seq = all_xpath_tags_seq[start_pos:end_pos]
        splited_xpath_subs_seq = all_xpath_subs_seq[start_pos:end_pos] 
        splited_labels_seq = all_labels_seq[start_pos:end_pos]

        
        # Если end_pos больше, чем позиция конца страницы, заполняем этот разрыв
        # в splited_-последовательностях PAD-токенами
        if end_pos >= len(all_token_ids_seq):
            flag = True
            current_len = len(splited_token_ids_seq)
            splited_token_ids_seq += [tokenizer.pad_token_id] * (max_length - current_len)
            splited_xpath_tags_seq += [padded_xpath_tags_seq] * (max_length - current_len)
            splited_xpath_subs_seq += [padded_xpath_subs_seq] * (max_length - current_len)
            splited_labels_seq += [-100] * (max_length - current_len)
            attention_mask = [1] * current_len + [0] * (max_length - current_len)
        else:
            # Иначе паддинг не нужен, splited_-последовательности по длине равны 384
            assert len(splited_token_ids_seq) == max_length
            attention_mask = [1] * max_length
        
        # features - список признаков чанков страницы
            
        # chunk_input_ids = torch.tensor(splited_token_ids_seq, dtype=torch.long)
        # attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        # chunk_token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        # chunk_xpath_tags_seq = torch.tensor(splited_xpath_tags_seq, dtype=torch.long)
        # chunk_xpath_subs_seq = torch.tensor(splited_xpath_subs_seq, dtype=torch.long)
        # chunk_labels = torch.tensor(splited_labels_seq, dtype=torch.long)
        # features.append(
        #     {
        #         'input_ids': chunk_input_ids,
        #         'token_type_ids': chunk_token_type_ids,
        #         'attention_mask': attention_mask,
        #         'xpath_tags_seq': chunk_xpath_tags_seq,
        #         'xpath_subs_seq': chunk_xpath_subs_seq,
        #         'labels': chunk_labels
        #     }
        # )
        features.append(
            {
                'input_ids': splited_token_ids_seq,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask,
                'xpath_tags_seq': splited_xpath_tags_seq,
                'xpath_subs_seq': splited_xpath_subs_seq,
                'labels': splited_labels_seq
            }
        )
        # features.append(
        #     SwdeFeature(
        #         input_ids=splited_token_ids_seq,
        #         token_type_ids=token_type_ids,
        #         attention_mask=attention_mask,
        #         xpath_tags_seq=splited_xpath_tags_seq,
        #         xpath_subs_seq=splited_xpath_subs_seq,
        #         labels=splited_labels_seq
        #     )
        # )
        
        start_pos = end_pos - doc_stride

        if flag:
            break

    return features

def chunk_df(dataset):
    features = []
    for page in dataset:
        features += chunk_page(page)
    print(type(features[0]))
    all_input_ids = torch.tensor([f['input_ids'] for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f['attention_mask'] for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f['token_type_ids'] for f in features], dtype=torch.long)
    all_xpath_tags_seq = torch.tensor([f['xpath_tags_seq'] for f in features], dtype=torch.long)
    all_xpath_subs_seq = torch.tensor([f['xpath_subs_seq'] for f in features], dtype=torch.long)
    all_labels = torch.tensor([f['labels'] for f in features], dtype=torch.long)


    dataset = SwdeDataset(all_input_ids=all_input_ids,
                          all_attention_mask=all_attention_mask,
                          all_token_type_ids=all_token_type_ids,
                          all_xpath_tags_seq=all_xpath_tags_seq,
                          all_xpath_subs_seq=all_xpath_subs_seq,
                          all_labels=all_labels)

    # chunked_dataset = {
    #     'input_ids': all_input_ids,
    #     'token_type_ids': all_token_type_ids,
    #     'attention_mask': all_attention_mask,
    #     'xpath_tags_seq': all_xpath_tags_seq,
    #     'xpath_subs_seq': all_xpath_subs_seq,
    #     'labels': all_labels
    # }

    # chunked_df = chunked_df.explode(chunked_df.columns.tolist())
    # chunked_df = pd.DataFrame.from_dict(chunked_dataset)
    # chunked_df = chunked_df.explode(chunked_df.columns.tolist())
    
    # chunked_df = Dataset.from_pandas(chunked_df)
    return dataset


In [ ]:
from transformers import MarkupLMProcessor

processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

train_dataset = MarkupLMDataset(data=train_data[0:100], processor=processor, max_length=max_length)
valid_dataset = MarkupLMDataset(data=valid_data[0:100], processor=processor, max_length=max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (9907 > 512). Running this sequence through the model will result in indexing errors


Сохранение датасетов

In [ ]:
torch.save(train_dataset, f'train_dataset_100.pth')
# t.save(train_info, f'data/train_info.pth')
torch.save(valid_dataset, f'valid_dataset_100.pth')
# t.save(test_info, f'data/test_info.pth')

In [ ]:
train_dataset_chunk = chunk_df(train_dataset)
valid_dataset_chunk = chunk_df(valid_dataset)

<class 'dict'>
<class 'dict'>


Сохранение датасетов, разбитых на чанки

In [1]:
torch.save(train_dataset_chunk, f'train_dataset_chunk_100.pth')
# t.save(train_info, f'data/train_info.pth')
torch.save(valid_dataset_chunk, f'valid_dataset_chunk_100.pth')

NameError: name 'torch' is not defined

In [19]:
example = train_dataset_chunk[1]
for k,v in example.items():
  print(k,v.shape)
  

AttributeError: 'tuple' object has no attribute 'items'

In [170]:
id2label = {0: "model", 1: "price", 2: "manufacturer", 3: "other"}
label2id = {label:id for id, label in id2label.items()}
for id, label in zip(example['input_ids'], example['labels']):
  if label != -100:
    print(processor.decode([id]), id2label[label])
  else:
    print(processor.decode([id]), label)

") tensor(-100)
 > tensor(-100)
 - tensor(-100)
1 tensor(-100)
) tensor(-100)
 { tensor(-100)

 tensor(-100)
	 tensor(-100)
	 tensor(-100)
	 tensor(-100)
	 tensor(-100)
if tensor(-100)
 ( tensor(-100)
pg tensor(-100)
Check tensor(-100)
. tensor(-100)
index tensor(-100)
Of tensor(-100)
(" tensor(-100)
qu tensor(-100)
=" tensor(-100)
) tensor(-100)
 > tensor(-100)
 - tensor(-100)
1 tensor(-100)
) tensor(-100)
 { tensor(-100)

 tensor(-100)
	 tensor(-100)
	 tensor(-100)
	 tensor(-100)
	 tensor(-100)
  tensor(-100)
  tensor(-100)
  tensor(-100)
 window tensor(-100)
. tensor(-100)
location tensor(-100)
 = tensor(-100)
 ' tensor(-100)
http tensor(-100)
:// tensor(-100)
mobile tensor(-100)
. tensor(-100)
buy tensor(-100)
. tensor(-100)
com tensor(-100)
/ tensor(-100)
ib tensor(-100)
uy tensor(-100)
/ tensor(-100)
Search tensor(-100)
Products tensor(-100)
. tensor(-100)
aspx tensor(-100)
? tensor(-100)
pg tensor(-100)
= tensor(-100)
0 tensor(-100)
& tensor(-100)
s tensor(-100)
=' tensor(-100)


KeyError: tensor(3)

## Create PyTorch Dataloaders

The next step is to create a PyTorch DataLoader, which allows us to get batches from the dataset.

Загружаем датасет из файлов

In [ ]:
train_dataset_chunk = torch.load('train_dataset_chunk.pth')
valid_dataset_chunk = torch.load('valid_dataset_chunk.pth')

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset_chunk, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(valid_dataset_chunk, batch_size=4, shuffle=True)

## Define model

We define the model here, which is a MarkupLM-base Transformer, with a token classifier head on top. The token classifier will have randomly initialized weights, while the base Transformer has pre-trained weights.



In [21]:

id2label = {0: "model", 1: "price", 2: "manufacturer", 3: "other"}
label2id = {label:id for id, label in id2label.items()}

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=id2label, label2id=label2id)

Some weights of MarkupLMForTokenClassification were not initialized from the model checkpoint at microsoft/markuplm-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also create a label_list, where each tag starts with a B (as seqeval expects the labels to be in IOB format).

In [22]:
label_list = ["B-" + x for x in list(id2label.values())]
label_list

['B-model', 'B-price', 'B-manufacturer', 'B-other']

We also define metric calculations (as we'd like to know the F1 score etc. during training). We'll use 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) for that, which is a library containing many tools for evaluating ML models.

In [23]:
import evaluate

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

## Train

Alright, let's train! Here we're training the model in native PyTorch, but of course you could also opt for things like 🤗 Accelerate, 🤗 Trainer, PyTorch Lightning,...

In [25]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

model.to(device)

model.train()
for epoch in range(2):  # loop over the dataset multiple times
    for batch in tqdm(train_dataloader):
        # get the inputs;
        # inputs = {k:v.to(device) for k,v in batch.items()}
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'xpath_tags_seq': batch[3],
                      'xpath_subs_seq': batch[4],
                      'labels': batch[5],
                      }

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch[5]
        preds, refs = get_labels(predictions, labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )

    train_metric = compute_metrics()
    wandb.log(train_metric)
    print(f"Epoch {epoch}:", train_metric)

  0%|          | 1/832 [00:00<08:20,  1.66it/s]

100%|██████████| 832/832 [05:38<00:00,  2.46it/s]


Epoch 0: {'manufacturer_precision': 0.972972972972973, 'manufacturer_recall': 0.6792452830188679, 'manufacturer_f1': 0.7999999999999999, 'manufacturer_number': 106, 'model_precision': 0.6962750716332379, 'model_recall': 0.7297297297297297, 'model_f1': 0.7126099706744867, 'model_number': 333, 'other_precision': 0.9979324872568397, 'other_recall': 0.9982829714999285, 'other_f1': 0.9981076986103615, 'other_number': 76877, 'price_precision': 0.7614678899082569, 'price_recall': 0.6916666666666667, 'price_f1': 0.7248908296943232, 'price_number': 120, 'overall_precision': 0.9962162301771785, 'overall_recall': 0.9962162301771785, 'overall_f1': 0.9962162301771785, 'overall_accuracy': 0.9962162301771785}


100%|██████████| 832/832 [05:38<00:00,  2.46it/s]


Epoch 1: {'manufacturer_precision': 1.0, 'manufacturer_recall': 1.0, 'manufacturer_f1': 1.0, 'manufacturer_number': 104, 'model_precision': 0.8161559888579387, 'model_recall': 0.9071207430340558, 'model_f1': 0.8592375366568915, 'model_number': 323, 'other_precision': 0.9995964975991607, 'other_recall': 0.9991127243395846, 'other_f1': 0.9993545524224453, 'other_number': 74385, 'price_precision': 1.0, 'price_recall': 1.0, 'price_f1': 1.0, 'price_number': 117, 'overall_precision': 0.9987187871184722, 'overall_recall': 0.9987187871184722, 'overall_f1': 0.9987187871184722, 'overall_accuracy': 0.9987187871184722}


In [26]:
train_metric

{'manufacturer_precision': 1.0,
 'manufacturer_recall': 1.0,
 'manufacturer_f1': 1.0,
 'manufacturer_number': 104,
 'model_precision': 0.8161559888579387,
 'model_recall': 0.9071207430340558,
 'model_f1': 0.8592375366568915,
 'model_number': 323,
 'other_precision': 0.9995964975991607,
 'other_recall': 0.9991127243395846,
 'other_f1': 0.9993545524224453,
 'other_number': 74385,
 'price_precision': 1.0,
 'price_recall': 1.0,
 'price_f1': 1.0,
 'price_number': 117,
 'overall_precision': 0.9987187871184722,
 'overall_recall': 0.9987187871184722,
 'overall_f1': 0.9987187871184722,
 'overall_accuracy': 0.9987187871184722}

In [29]:
metric = evaluate.load("seqeval")

model.eval()
for batch in eval_dataloader:
    # batch = {k: v.to(device) for k, v in batch.items()}
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'xpath_tags_seq': batch[3],
                'xpath_subs_seq': batch[4],
                'labels': batch[5],
                }


    with torch.no_grad():
        outputs = model(**inputs)


    predictions = outputs.logits.argmax(dim=-1)
    labels = batch[5]

    preds, refs = get_labels(predictions, labels)

    metric.add_batch(
        predictions=preds,
        references=refs,
    )


eval_metrics = metric.compute()

In [30]:
eval_metrics

{'manufacturer': {'precision': 0.8365019011406845,
  'recall': 0.6875,
  'f1': 0.7547169811320755,
  'number': 320},
 'model': {'precision': 0.3312597200622084,
  'recall': 0.6474164133738601,
  'f1': 0.43827160493827155,
  'number': 329},
 'other': {'precision': 0.9949957536961107,
  'recall': 0.993181838898156,
  'f1': 0.9940879688341072,
  'number': 109707},
 'price': {'precision': 0.46078431372549017,
  'recall': 0.4144620811287478,
  'f1': 0.43639740018570095,
  'number': 567},
 'overall_precision': 0.9883162193593754,
 'overall_recall': 0.9883162193593754,
 'overall_f1': 0.9883162193593754,
 'overall_accuracy': 0.9883162193593754}

In [15]:
# res = {}

# for x in all_refs:
#     for y in x:
#         y = int(y)
#         if y in res:
#             res[y] += 1
#         else:
#             res[y] = 1
# print(res)

In [32]:
from collections import Counter

count = {
    0: 0,
    1: 0,
    2: 0,
    3: 0
}

for example in train_data:
    example_cnt = Counter(example["node_labels"])
    # print(example_cnt)
    for key in count.keys():
        count[key] += example_cnt[key]

print(count)

TypeError: unhashable type: 'list'

## Inference

Let's try out the model on a new web page for which we have the nodes and xpaths. Here we'll just use one of our training set.

In [33]:
type(data)

dict

In [34]:
data['amazon'][0].keys()

dict_keys(['nodes', 'xpaths', 'node_labels'])

In [35]:
nodes = data['jr'][50]['nodes']
xpaths = data['jr'][50]['xpaths']
node_labels = data['jr'][50]['node_labels']
print("Nodes:", nodes)
print("Xpaths:", xpaths)

Nodes: [['Vivitar Vivicam 7022 7MP Digital Camera / GREEN in Point and Shoot Digital Cameras at JR.com', '[if lt IE 7]>\n\t<link href="http://www.jr.com/assets/gui/1.2.6/src/jrui/build/menu/skins/clint/menu-ie6.css" type="text/css" rel="stylesheet"/>\n\n\t<link rel="stylesheet" type="text/css" href="/media/styles/ie6_v1_m56577569833774833.css" />\n<![endif]', 'Link to the top', 'My Account', '-', 'Login', 'Order Status', 'NYC Stores', 'Email Newsletter', 'CATEGORIES', 'BRANDS', 'SPECIALTY STORES', 'SAVINGS', 'PROMOTIONS', 'LEARN', 'HELP', 'Order By Phone:', '800-806-1115', 'Customer Service:', '800-426-6027', 'Corporate Sales:', '800-221-3191', '0 items', '$0.00', 'Search:', 'BEGIN BREADCRUMB', 'Home', '>', 'Cameras', '>', 'Digital Cameras', '>', 'Point and Shoot', '> Vivitar Vivicam 7022 7MP Digital Camera / GREEN', 'END BREADCRUMB', '<!--\nfunction doProductPost(formAction, template, qs) {\ndocument.navigate.nav_action.value = formAction;\ndocument.navigate.nav_template.value = templ

In [53]:
node_labels

[[3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  3,
  1,
  3,
  1,
  1,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,


In [27]:
data['jr'][50]

{'nodes': [['Vivitar Vivicam 7022 7MP Digital Camera / GREEN in Point and Shoot Digital Cameras at JR.com',
   '[if lt IE 7]>\n\t<link href="http://www.jr.com/assets/gui/1.2.6/src/jrui/build/menu/skins/clint/menu-ie6.css" type="text/css" rel="stylesheet"/>\n\n\t<link rel="stylesheet" type="text/css" href="/media/styles/ie6_v1_m56577569833774833.css" />\n<![endif]',
   'Link to the top',
   'My Account',
   '-',
   'Login',
   'Order Status',
   'NYC Stores',
   'Email Newsletter',
   'CATEGORIES',
   'BRANDS',
   'SPECIALTY STORES',
   'SAVINGS',
   'PROMOTIONS',
   'LEARN',
   'HELP',
   'Order By Phone:',
   '800-806-1115',
   'Customer Service:',
   '800-426-6027',
   'Corporate Sales:',
   '800-221-3191',
   '0 items',
   '$0.00',
   'Search:',
   'BEGIN BREADCRUMB',
   'Home',
   '>',
   'Cameras',
   '>',
   'Digital Cameras',
   '>',
   'Point and Shoot',
   '> Vivitar Vivicam 7022 7MP Digital Camera / GREEN',
   'END BREADCRUMB',
   '<!--\nfunction doProductPost(formAction, tem

In [36]:
from collections import Counter

count = {
    0: 0,
    1: 0,
    2: 0,
    3: 0
}
example = data['jr'][50]
example_cnt = Counter(example["node_labels"][0])
# print(example_cnt)
for key in count.keys():
    count[key] += example_cnt[key]

print(count)

{0: 2, 1: 9, 2: 1, 3: 354}


We'll prepare the example for the model using the processor. Note that we're passing `return_offsets_mapping=True`, as the offsets allow us to determine which tokens are at the start of a given word at which aren't.

In [1]:
# prepare for model
# note that you don't need to prepare node_labels, we just have them available here so we'll compare to the ground truth
# print(processor.max_length)
encoding = processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, truncation=True, max_length=max_length, return_offsets_mapping=True, return_tensors="pt").to(device)
for k,v in encoding.items():
  print(k,v.shape)

NameError: name 'processor' is not defined

Let's perform a forward pass:

In [38]:
# we don't need the offset mapping and labels for the forward pass
offset_mapping = encoding.pop("offset_mapping")
labels = encoding.pop("labels")

# forward pass
with torch.no_grad():
  outputs = model(**encoding)

../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [101,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [10

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


The model outputs logits of shape (batch_size, seq_len, num_labels). We just take the highest logit (score) per token as prediction:

In [45]:
predictions = outputs.logits.argmax(-1)
print(predictions)

tensor([[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3

In [46]:
print(example['node_labels'][0])

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 1, 3, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

The model makes predictions at the token level, however we're only interested in the predicted label for the first token of each node.

This can be achieved by accessing the word_ids (to know whether or not the token is a special token or not) and the offset_mapping (to know whether or not the token is the first of a particular node).

In [33]:
results = {"Node": [], "Predicted": [], "Ground truth": []}

for pred_id, word_id, offset, label_id in zip(predictions[0].tolist(), encoding.word_ids(0), offset_mapping[0].tolist(), labels[0].tolist()):
  if word_id is not None and offset[0] == 0:
    print(f"Node: {nodes[0][word_id]}")
    print(f"Predicted: {id2label[pred_id]}")
    print(f"Ground truth: {id2label[label_id]}")
    print("----------")
    results["Node"].append(nodes[0][word_id])
    results["Predicted"].append(id2label[pred_id])
    results["Ground truth"].append(id2label[label_id])

Node: Vivitar Vivicam 7022 7MP Digital Camera / GREEN in Point and Shoot Digital Cameras at JR.com
Predicted: other
Ground truth: other
----------
Node: [if lt IE 7]>
	<link href="http://www.jr.com/assets/gui/1.2.6/src/jrui/build/menu/skins/clint/menu-ie6.css" type="text/css" rel="stylesheet"/>

	<link rel="stylesheet" type="text/css" href="/media/styles/ie6_v1_m56577569833774833.css" />
<![endif]
Predicted: other
Ground truth: other
----------
Node: Link to the top
Predicted: other
Ground truth: other
----------
Node: My Account
Predicted: other
Ground truth: other
----------
Node: -
Predicted: other
Ground truth: other
----------
Node: Login
Predicted: other
Ground truth: other
----------
Node: Order Status
Predicted: other
Ground truth: other
----------
Node: NYC Stores
Predicted: other
Ground truth: other
----------
Node: Email Newsletter
Predicted: other
Ground truth: other
----------
Node: CATEGORIES
Predicted: other
Ground truth: other
----------
Node: BRANDS
Predicted: other
Gr

Let's pretty print the results as a Pandas dataframe:

In [35]:
import pandas as pd

pd.DataFrame.from_dict(results)

,Node,Predicted,Ground truth
0,Vivitar Vivicam 7022 7MP Digital Camera / GREE...,other,other
1,"[if lt IE 7]>\n\t<link href=""http://www.jr.com...",other,other
2,Link to the top,other,other
3,My Account,other,other
4,-,other,other
5,Login,other,other
6,Order Status,other,other
7,NYC Stores,other,other
8,Email Newsletter,other,other
9,CATEGORIES,other,other


In [36]:
results

{'Node': ['Vivitar Vivicam 7022 7MP Digital Camera / GREEN in Point and Shoot Digital Cameras at JR.com',
  '[if lt IE 7]>\n\t<link href="http://www.jr.com/assets/gui/1.2.6/src/jrui/build/menu/skins/clint/menu-ie6.css" type="text/css" rel="stylesheet"/>\n\n\t<link rel="stylesheet" type="text/css" href="/media/styles/ie6_v1_m56577569833774833.css" />\n<![endif]',
  'Link to the top',
  'My Account',
  '-',
  'Login',
  'Order Status',
  'NYC Stores',
  'Email Newsletter',
  'CATEGORIES',
  'BRANDS',
  'SPECIALTY STORES',
  'SAVINGS',
  'PROMOTIONS',
  'LEARN',
  'HELP',
  'Order By Phone:',
  '800-806-1115',
  'Customer Service:',
  '800-426-6027',
  'Corporate Sales:',
  '800-221-3191',
  '0 items',
  '$0.00',
  'Search:',
  'BEGIN BREADCRUMB',
  'Home',
  '>',
  'Cameras',
  '>',
  'Digital Cameras',
  '>',
  'Point and Shoot',
  '> Vivitar Vivicam 7022 7MP Digital Camera / GREEN',
  'END BREADCRUMB',
  '<!--\nfunction doProductPost(formAction, template, qs) {\ndocument.navigate.nav_a